In [32]:
import sys

from cobra.io import read_sbml_model

sys.path.insert(0, r"C:\Users\Bisbii\PythonProjects\ExpAlgae\src")
from ExpAlgae.model.COBRAmodel import *
from cobra.flux_analysis import pfba, find_blocked_reactions, flux_variability_analysis as fva

In [33]:
data_directory = "../data"
model = MyModel(join(data_directory, "models/model_with_trials.xml"), "e_Biomass__cytop")
model.add_medium(join(data_directory, "media.xlsx"), "base_medium")
model.setup_condition("4")

Loading

Reactions: 4584
Metabolites: 3655
Genes: 1684
Model loaded


In [35]:
model.metabolites.C02094__chlo.summary()

Percent,Flux,Reaction,Definition
100.00%,0.006754,R03824__chlo,C05435__chlo --> C02094__chlo
Percent,Flux,Reaction,Definition
4.87%,-0.0003291,R00032__chlo,C00007__chlo + C02094__chlo --> 2.0 C00376__chlo
9.26%,-0.0006255,R07558__chlo,C00007__chlo + 2.0 C00080__chlo + 2.0 C00138__chlo + C02094__chlo --> C00001__chlo + 2.0 C00139__chlo + C08591__chlo
60.52%,-0.004088,R09747__chlo,2.0 C00007__chlo + 4.0 C00080__chlo + 4.0 C00138__chlo + C02094__chlo --> 2.0 C00001__chlo + 4.0 C00139__chlo + C06098__chlo
25.27%,-0.001707,R10282__chlo,C02094__chlo --> C20484__chlo
0.07%,-4.967E-06,e_Pigment_trial4__chlo,0.0013 C02094__chlo + 0.0077 C05306__chlo + 0.0048 C05307__chlo + 0.0008 C05432__chlo + 0.0253 C05433__chlo + 0.0162 C06098__chlo + 0.0014 C08579__chlo + 0.1637 C08591__chlo + 0.0025 C08601__chlo + 0.0031 C08606__chlo + 1.0492 C08614__chlo + 0.4467 C20484__chlo --> e_Pigment__chlo


In [ ]:
model.metabolites.C00065__mito.summary(fva=1.0)

In [ ]:
with model:
    # model.create_sink("C00148__mito")
    print(fva(model, ['R02236__cytop'], fraction_of_optimum=0.0))

In [ ]:
model.test_reaction("R02236__cytop")

In [ ]:
model.metabolites.C00025__mito.summary(fva = 1.0)

In [ ]:
model.summary()

In [ ]:
#create copy reactions
with model:
    carbohydrate_copy = model.reactions.e_Carbohydrate__cytop.copy()
    carbohydrate_copy.id = 'e_Carbohydrate_no_starch__cytop'
    biomass_copy = model.reactions.e_Biomass__cytop.copy()
    biomass_copy.id = 'e_ActiveBiomass__cytop'
    model.add_reactions([carbohydrate_copy, biomass_copy])
    model.objective = 'e_ActiveBiomass__cytop'
    model.reactions.e_Carbohydrate__cytop.bounds = (0, 0)
    #remove starch from e_carbohydrate
    carbohydrate_copy.add_metabolites({model.metabolites.C00369__chlo: -carbohydrate_copy.metabolites[model.metabolites.C00369__chlo]})

    #adjust other carbohydrates
    carbohydrate_copy.add_metabolites({model.metabolites.C00052__cytop:  -3.35762291 - carbohydrate_copy.metabolites[model.metabolites.C00052__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00096__cytop:  -1.58181346 - carbohydrate_copy.metabolites[model.metabolites.C00096__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00935__cytop:  -2.52493299 - carbohydrate_copy.metabolites[model.metabolites.C00935__cytop]})

    carbohydrate_copy.add_metabolites({model.metabolites.C00015__cytop:  5.8825559 - carbohydrate_copy.metabolites[model.metabolites.C00015__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00035__cytop:  1.58181346 - carbohydrate_copy.metabolites[model.metabolites.C00035__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00001__cytop:  7.46436936 - carbohydrate_copy.metabolites[model.metabolites.C00001__cytop]})

    print(carbohydrate_copy.metabolites)

    biomass_copy.add_metabolites({model.metabolites.e_Carbohydrate__cytop:  -0.203306526 - biomass_copy.metabolites[model.metabolites.e_Carbohydrate__cytop]})
    print(biomass_copy.metabolites)
    print(model.optimize().objective_value*0.7083)

    # total = abs(sum([biomass_copy.metabolites[met] for met in biomass_copy.reactants if biomass_copy.metabolites[met] > -10]))
    # for  met in biomass_copy.reactants:
    #     if biomass_copy.metabolites[met] > -10:
    #         print(met.id)
    #         new_st = biomass_copy.metabolites[met]/total
    #         print(new_st)
    #         print(biomass_copy.metabolites[met])
    #         biomass_copy.add_metabolites({model.metabolites.get_by_id(met.id):  new_st - biomass_copy.metabolites[met]})
    #         print(biomass_copy.metabolites[met])
    # print(sum([biomass_copy.metabolites[met] for met in biomass_copy.reactants if biomass_copy.metabolites[met] > -10]))
    # print(model.optimize().objective_value)
    model.write(os.path.join(data_directory, "models/model_dfba.xml"))

In [ ]:
data_directory = "../data"
model = MyModel(join(data_directory, "model_dfba.xml"), "e_Biomass_trial4__cytop")
model.add_medium(join(data_directory, "media.xlsx"), "base_medium")
model.setup_condition("4")

In [ ]:
pfba_sol = pfba(model)
model.summary(pfba_sol)

In [ ]:
with model:
    model.reactions.DM_C00369__chlo.bounds = (0.00147, 1000)
    pfba_sol = pfba(model)
    # print(model.metabolites.C00369__chlo.summary(pfba_sol))
    print(model.summary(pfba_sol))

In [ ]:
sum([model.reactions.e_Biomass_trial4__cytop.metabolites[reactant] for reactant in model.reactions.e_Biomass_trial4__cytop.metabolites if model.reactions.e_Biomass_trial4__cytop.metabolites[reactant] < 0 and model.reactions.e_Biomass_trial4__cytop.metabolites[reactant]>-1])

In [ ]:
original_sol = model.optimize().objective_value

In [ ]:
model.summary()

In [ ]:
model.metabolites.C00116__cytop.formula_weight

In [ ]:
fva(model, ["T_SUCBZth__chlomem"])

In [ ]:
with model:
    # model.biomass_reaction = model.bio_reaction.id
    # model.reactions.e_Carbohydrate__cytop.add_metabolites({model.metabolites.C00369__chlo: -model.reactions.e_Carbohydrate__cytop.metabolites[model.metabolites.C00369__chlo]})
    # model.reactions.e_Biomass_trial4__cytop
    # model.reactions.e_Acid__cytop.add_metabolites({model.metabolites.C00116__cytop: -model.reactions.e_Acid__cytop.metabolites[model.metabolites.C00116__cytop]})
    # model.reactions.DM_C00369__chlo.bounds = (0.305381773/model.metabolites.C00369__chlo.formula_weight*1000*original_sol, 1000)
    # model.reactions.DM_C00369__chlo.bounds = (0.00147, 1000)
    # [setattr(x, 'objective_coefficient', 0) for x in model.reactions if x.objective_coefficient != 0]
    # model.reactions.EX_e_Biomass__dra.objective_coefficient = 0.7
    # model.reactions.DM_C00369__chlo.objective_coefficient = 0.3
    # model.objective = "e_Biomass__cytop"
    sol = pfba(model)
    print(model.summary(solution=sol))
    # print(model.metabolites.C00369__chlo.summary(sol))

In [ ]:
total = sum([reactant for reactant in model.reactions.e_Biomass__cytop.metabolites if model.reactions.e_Biomass__cytop.metabolites[reactant] < 0 and model.reactions.e_Biomass__cytop.metabolites[reactant]>-1])
for reactant in model.reactions.e_Biomass__cytop.reactants:
    st =

model.reactions.DM_C00369__chlo

model.reactions.e_Lipid__cytop.add_metabolites({model.metabolites.C00422__lip: 0.13})

model.reactions.e_Lipid__cytop.metabolites

model.reactions.e_Carbohydrate__cytop.metabolites

0.3212*(1-(0.0123373*48660.08/1000*0.50869+0.13*904.7821/1000*0.11407))

(1-(0.0123373*48660.08/1000*0.50869+0.13*904.7821/1000*0.11407))

model.reactions.DM_C00369__chlo.bounds = (0.3/48660.08*1000*0.21, 1000)

model.reactions.DM_C00369__chlo

model.summary()